In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
from basis_generator import generate_basis, sel_seq

In [2]:
dim = 2
num_obs = 3
len_seq = 2
out_max = 1
basis_size= 50
seq_method="sel_sequences"

In [3]:
LG_events = [((0, 0), (0, 1)),
            ((1, 1), (0, 1)),
            ((0, 0), (1, 2)),
            ((1, 1), (1, 2)),
            ((0, 1), (0, 2)),
            ((1, 0), (0, 2)),
                ]

sequences = sel_seq(num_obs = num_obs,
                len_seq = len_seq,
                out_max = out_max,
                remove_last_out = False)

n = len(sequences)

In [4]:
X_basis, rank = generate_basis(dim = dim,
                num_obs = num_obs,
                len_seq = len_seq,
                out_max = out_max,
                batch_size = basis_size,
                seq_method= seq_method,
                sel_sequences = [len_seq],
                remove_last_out = False)


In [5]:
alpha = cp.Variable((len(X_basis), 1))

C = np.zeros((n,n))
for i, seq_row in enumerate(sequences):
    if seq_row in LG_events:
        C[i,i] = 1

X = cp.Variable((n,n), symmetric=True)
constraints = [X >> 0]
constraints += [
    X == sum([alpha[i]*X_basis[i] for i in range(len(X_basis))])
]
constraints += [
    cp.trace(X) == num_obs**len_seq
]

prob = cp.Problem(cp.Maximize(2*cp.trace(C@X)-3),
                  constraints)

prob.solve(solver=cp.MOSEK, verbose=False)

print(prob.status)
print("The optimal value is", prob.value)

optimal
The optimal value is 1.4999999999918119
